# Visit the Wikipedia hyperlinks graph!
In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we are curious to rank the articles according to some criteria. 

In [1]:
import pandas as pd
import json
import pickle
from tqdm import tqdm

from collections import defaultdict
from heapq import *
import numpy as np
from tqdm import tqdm

import networkx as nx

## Research questions


### **[RQ1]** 
Build the graph <img src="https://latex.codecogs.com/gif.latex?G=(V,&space;E)" title="G=(V, E)" /> where *V* is the set of articles and *E* the hyperlinks among them, and provide its basic information:
 
- If it is direct or not
- The number of nodes
- The number of edges 
- The average node degree. Is the graph dense?

###### Build the graph!

In [2]:
F = open('wiki-topcats-reduced.txt','r') 
rows=F.read().split('\n') #split the rows
grafo={}#initialize the graph
reciver_nodes=set() #create a set of the target of the edges
for row in rows:
        link=row.split('\t') 
        if link[0] not in grafo: #add the vertex if it doesn't exist
            try:
                grafo[link[0]]=set()
                grafo[link[0]].add(link[1]) #add the edge
                reciver_nodes.add(link[1])
            except: print('empty row')
        else:
            grafo[link[0]].add(link[1])
            reciver_nodes.add(link[1])
F.close()

empty row


We need to add to our graph nodes that only recives edges.

In [3]:
onlyreciver=(reciver_nodes-set(grafo.keys()))
for node in onlyreciver:
    grafo[node]=set()

###### Find out if it's directed or not:

In [4]:
for neighbors in grafo['52']:
    print('52' in grafo[neighbors])

False
False
False


We see that it's directed, since we have unspecular edges.

###### Get the number of nodes!

In [5]:
nodes=len(grafo)
nodes

461194

###### Get the number of edges!

In [6]:
edges=0
for node in grafo:
    try: edges+=(len(grafo[node]))
    except: pass
edges

2645247

###### Get the average node degree. Is the graph dense?

In graph theory, the degree of a vertex of a graph is the number of edges incident to the vertex. The degree of a vertex $v$ is denoted $\deg(v)$.

The average degree is denoted as $\frac{E}{N}$:


In [7]:
avg_degree= edges/nodes
avg_degree

5.735649206190887

As we see, the average node degree is slightly great than six.
In mathematics, a dense graph is a graph in which the number of edges is close to the maximal number of edges.
Looking at our numbers, we can immediatly say that the graph is not dense, but very sparse.

To have a matematical confirm of this we can compute the density as $D={\frac{|E|}{|N|\,(|N|-1)}}$

In [8]:
density = edges/(nodes*(nodes-1))
density

1.2436548703451455e-05

Our hypothesis is verified.

For completeness we create a dictionary that maps every page with it's name:

In [9]:
F = open('wiki-topcats-page-names.txt','r')
articles={}
for line in F.readlines():
    num=line.split()[0]
    tit=line.split()[1:]
    title=' '.join(tit)
    articles[num]=title
F.close()

### **[RQ2]** 
Given a category <img src="https://latex.codecogs.com/gif.latex?C_0&space;=&space;\{article_1,&space;article_2,&space;\dots&space;\}" title="C_0 = \{article_1, article_2, \dots \}" /> as input we want to rank all of the nodes in *V* according to the following criteria:
	
* Obtain a *block-ranking*, where the blocks are represented by the categories. In particular, we want:

<img src="https://latex.codecogs.com/gif.latex?block_{RANKING}&space;=\begin{bmatrix}&space;C_0&space;\\&space;C_1&space;\\&space;\dots&space;\\&space;C_c\\&space;\end{bmatrix}" title="block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}" />
	
Each category $C_i$ corresponds to a list of nodes. 

The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

<img src="https://latex.codecogs.com/gif.latex?$$distance(C_0,&space;C_i)&space;=&space;median(ShortestPath(C_0,&space;C_i))$$" title="distance(C_0, C_i) = median(ShortestPath(C_0, C_i))" />

The lower is the distance from $C_0$, the higher is the $C_i$ position in the rank. $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$  and $C_i$. Moreover, the length of a path is given by the sum of the weights of the edges it is composed by.



###### Creating the categories dictionary

We create a dictionary that associate evry category with his articles.

In [4]:
F = open('wiki-topcats-categories.txt','r')
categorie={}
for line in F.readlines():
    riga=line.split(' ')
    categoria=(riga[0].replace('Category:','').replace(';',''))
    articles=(riga[1:-1])
    articles.append(riga[-1].replace('\n',''))
    categorie[categoria]= articles
F.close()

We must take into account all the categories that has a number of articles greater than 3500, so we clean up:

In [5]:
categories={}
for cat in categorie:
    if len(categorie[cat])>=3500:
        categories[cat]=categorie[cat]

We remove every article that is not in our graph:

In [9]:
for cat in categories:
    toremove=[]
    for art in categories[cat]:
        if art not in grafo:
            toremove.append(art)
    for art in toremove:
        categories[cat].remove(art)

        

We store our final categories dictionary in an external pickle file so we can reload it in need.

In [12]:
with open('categories.pickle', 'wb') as handle:
    pickle.dump(categories, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [104]:
with open('categories.pickle', 'rb') as handle:
    categories = pickle.load(handle)

In [98]:
len(categories)

35

###### The block ranking

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/DijkstraDemo.gif/220px-DijkstraDemo.gif)

We can try to solve the problem with dijkstra algorithm:

In [13]:
#distance between two nodes:

def dijkstra(graph, f, t): #graph, from, to
    #sorting the graph
    g = defaultdict(list, grafo)
    q= [(0,f)] #intializing the queque
    seen=set() #initializing the set of seen nodes
    distances = {f: 0} #initializing the dict of the distances
    while q: #we run the algo until the queue is not empty
        (cost,v1) = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            if v1 == t: return (cost) #if we reach the target, we stop
            if g.get(v1) is not None:  #if the vertex that we are examining has children, we continue our search
                for v2 in g.get(v1):
                    if v2 in seen: continue
                    prev = distances.get(v2, None)
                    new = cost + 1 #increasing the distance
                    if prev is None or new < prev:
                        distances[v2] = new
                        heappush(q, (new, v2))

    return float("inf")


In [14]:
#distance between categories

def catdistance(graph, catdict, cat1, cat2): #graph, dictionary of categories, and two categories
    cat1nodes= catdict[cat1]
    cat2nodes= catdict[cat2]
    distances=[]
    for node in cat1nodes:
        for node2 in cat2nodes:
            distances.append(dijkstra(graph=graph, f=node, t=node2))
    return np.median(distances)

In [15]:
catdistance(cat1='Japanese_rock_music_groups', cat2='Visual_kei_bands', catdict=categories, graph=grafo)


2.5

This works but is reeeeally slow. We want something more efficient. We can try a different approach using BFS algorithm, since the edges are not weighted.

BFS should be faster since his running time is something like $	\mathcal{O}{\bigl (}N+E{\bigr )} $, while dijkstra's is ${\displaystyle \mathcal{O}{\bigl (}E+N^{2}{\bigr )}=\mathcal{O}{\bigl (}N^{2}{\bigr )}}$ .

Morover, we can reduce drastically our runningtime creating a dictionary that associate every node of the graph with the distances from every node of the input category. 

![BFS](https://sadmanamin.com/wp-content/uploads/2017/07/videotogif_2017.07.01_20.55.06.gif)

Creating a function that takes a graph and a node as input and return a dictionary of his distance from every other node:

In [80]:
def bfs(graph, node):
    shortestpath={} #the dict 
    visited=set() #set of vertex that we have already seen
    queue=[node] #a queue of node that we have to examine
    dist=0 #at first we are at distance 0 from the node, we increase this count as we step away
    while queue: #if the queue is not empty
        vicini = [] #initialize an empty list of neighbours of the nodes
        for vertex in queue: #for each vertex of a given distance from the node
            if vertex not in visited: 
                visited.add(vertex) #we add to the set of the visited nodes
                neighbours = graph[vertex] #we add his neihgbours to the list
                vicini.extend(neighbours - visited) #we don't want to have duplicates
                shortestpath[vertex] = dist #we add the node to the dictionary
        queue=vicini #we reset the queue as the list of neighbours, that we are going to analize next
        dist += 1 #we increase the distance, and we are ready to repeat the cycle
    return shortestpath #we finally have our dictionary

Now we want a function that collects the bfs that starts from every node of the input category.

We also want to store this list in an external file since it's a long operation and we don't want to repeat it.

In [88]:
def solutiondata(graph,inp_cat, catdict): #input: a graph, an input category and a dictionary of categories
    inpnodes=catdict[inp_cat] #nodes of the input category
    bfssol=[] #the list of dictionary of distances from every point of the graph
    for node in tqdm(inpnodes):
        bfssol.append(bfs(grafo, node)) 
    with open('bfssol.pickle', 'wb') as handle: #store the list in an external file
        pickle.dump(bfssol, handle, protocol=pickle.HIGHEST_PROTOCOL)


Solving the task:

In [10]:
inputcat=input()

Year_of_birth_unknown


In [ ]:
solutiondata(catdict=categories, graph=grafo, inp_cat=inputcat) #we create our list of dictionaries

In [12]:
with open('bfssol.pickle', 'rb') as handle:
    bfssol = pickle.load(handle)

Now we want to create a dict (in some sense an 'inverted index') that has as keys every node of the graph and as values every shortespath from the nodes of the input category (now we have as keys the node of the input category)

In [13]:
finaldict = {}

for diz in (bfssol):
    for vert, value in diz.items():
        if vert not in finaldict:
            finaldict[vert] = [value] 
        else:
            finaldict[vert].append(value)

In [14]:
with open('finaldict.pickle', 'wb') as handle:
    pickle.dump(finaldict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [12]:
with open('finaldict.pickle', 'rb') as handle:
    finaldict = pickle.load(handle)

Now we can finally build our blockranking:

In [9]:
blockranking=[(0, inputcat)]
blockranking

[(0, 'Year_of_birth_unknown')]

In [10]:
lung= len(categories[inputcat]) #we will need this to check how many impossible paths we have for each pair 
                                #between the input cat and the others
for cat in tqdm(categories): #for every category except the input one
    if cat!=inputcat:
        paths=[] #every path between category nodes and input cat nodes
        catnodes=categories[cat]
        for node in catnodes:
            if node in finaldict: #if at least a path exist
                paths+=(finaldict[node]) #we add every path
                imppaths= lung-len(finaldict[node])
                paths+=([float('inf')]*imppaths) #we add the impossible paths marking them with "infinite" distance
            else:
                paths+=([float('inf')]*lung) #if we have no path, we add only impossible ones.
        median=np.median(paths) #the median of the path
        if median==float('inf'): #we want something that differenciate the distances that have as a median an infinite,
                                 #so we set the median as 10000 plus the number of impossible paths
            median=10000+paths.count(float('inf'))
        blockranking.append(( median, cat)) #we append the category to the blockranking


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [01:57<00:00,  1.11it/s]


In [25]:
blockranking=(sorted(blockranking))

In [28]:
with open('blockranking.pickle', 'wb') as handle:
    pickle.dump(blockranking, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [11]:
with open('blockranking.pickle', 'rb') as handle:
    blockranking = pickle.load(handle)

The final blockranking:

In [12]:
blockranking

[(0, 'Year_of_birth_unknown'),
 (6.0, 'American_film_actors'),
 (6.0, 'American_films'),
 (6.0, 'American_television_actors'),
 (6.0, 'British_films'),
 (6.0, 'English-language_films'),
 (6.0, 'English_television_actors'),
 (7.0, 'American_Jews'),
 (7.0, 'Article_Feedback_Pilot'),
 (7.0, 'Black-and-white_films'),
 (7.0, 'Indian_films'),
 (7.0, 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies'),
 (7.0, 'People_from_New_York_City'),
 (8.0, 'Debut_albums'),
 (8.0, 'English-language_albums'),
 (8.0, 'Fellows_of_the_Royal_Society'),
 (8.0, 'Rivers_of_Romania'),
 (9.0, 'Place_of_birth_missing_(living_people)'),
 (9.0, 'Windows_games'),
 (10.0, 'American_military_personnel_of_World_War_II'),
 (10.0, 'Living_people'),
 (10.0, 'The_Football_League_players'),
 (11.0, 'English_cricketers'),
 (11.0, 'Harvard_University_alumni'),
 (13.0, 'English_footballers'),
 (4971590, 'Association_football_goalkeepers'),
 (5502511, 'Association_football_defenders'),
 (5657119, 'Major_League_


Now that we obtained the $block_{RANKING}$ vector, we want to sort the nodes in each category. That's how we are going to do:




Suppose the categories order, given from the previous point, is $C_0, C_1, C_2$


__[STEP1]__ Compute subgraph induced by $C_0$. For each node compute the sum of the weigths of the in-edges.

 <img src="https://latex.codecogs.com/gif.latex?score_{article_i}&space;=&space;\sum_{i&space;\in&space;in-edges}&space;w_i" title="score_{article_i} = \sum_{j \in in-edges(article_i)} w_j" />

__[STEP2]__ Extend the graph to the nodes that belong to $C_1$. Thus, for each article in $C_1$ compute the score as before. __Note__ that the in-edges coming from the previous category, $C_0$, have as weights the score of the node that sends the edge.


__[STEP3]__ Repeat Step2 up to the last category of the ranking. 


If an article is in more than one category, we want him only in the nearest category to the input category.

In [105]:
visti= set() #a set of already seen vectors
for el in tqdm(blockranking): #every category in order
    cat=el[1] 
    nodi=[]
    nodi+=categories[cat]
    for node in nodi: #for every node of the category
        if node not in visti: #if is the first time that we meet him
            visti.add(node) #we add it to the set
        else: 
            categories[cat].remove(node) #we remove them from the category.

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [02:26<00:00,  4.20s/it]


In [107]:
with open('categoriesclean.pickle', 'wb') as handle:
    pickle.dump(categories, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [108]:
with open('categoriesclean.pickle', 'rb') as handle:
    categories = pickle.load(handle)

In [14]:
#we introduce the library networkx, in order to have the possibility to do some operation with the helpful directed graphs class
G=nx.DiGraph(incoming_graph_data=grafo) 

__[STEP1]__

In [135]:
subnodes=[] #nodes of the subgraph

In [136]:
subnodes+=categories[inputcat]

In [137]:
sub=G.subgraph(nodes=subnodes) #creating the first subgraph

In [138]:
pesi={} #a dict with weights of each node
ranking=[] #the ordered list of the ranking of each node

In [139]:
rank=[] 
# in_degree returns a dictionary with nodes as keys and in-degree as values
# (The node in-degree is the number of edges pointing in to the node) 
for node in sub.in_degree: #for each node and starting weigth
    nodo=node[0] 
    peso=node[1]
    pesi[nodo]=peso #we set his weight in the dict
    rank.append((peso, nodo)) #we append him to rank list
for node in sorted(rank, reverse=True): #for each node, starting from the highest rank
    ranking.append(node[1]) #we append the node to the final list of rankings

__[STEP2&3]__

In [140]:
for el in tqdm(blockranking[1:]): #for every category(except the input one)
    rank=[] #we inialize the list of ranks of this category
    cat=el[1]
    catnodes=categories[cat]
    subnodes+=catnodes
    sub=G.subgraph(nodes=subnodes) #we add the nodes to the subgraph
    for node in catnodes: #for each node of the category
        peso=0 #the weigth is 0 at first 
        for pred in sub.predecessors(node): #predecessors return a list of predecessor nodes of n.
            if pred in pesi: #if comes from an upper category
                peso+=pesi[pred] #we add the weight in the dictionary
            else: peso+=1
        rank.append((peso, node)) #we append the node to the rank list 
    for node in sorted(rank, reverse=True): #for each node, starting from the highest rank
        ranking.append(node[1]) #we append the node to the final list of rankings
        pesi[node[1]]=node[0] #we update our dictionary of weights


100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:12<00:00,  3.15it/s]


Finished! Now let's take a look to the highest and lowest articles in our ranking list:

In [144]:
for node in ranking[0:20]:
    print(articles[node])

Diogenes Lartius
Pausanias (geographer)
Theocritus
Dong Zhuo
L Bu
Yuan Shu
Stobaeus
Penda of Mercia
Stephen Dingate
Zhang Fei
Sextus Empiricus
Stigand
Ivan Alexander of Bulgaria
Tancred of Hauteville
Pope Nicholas II
Sweyn II of Denmark
Amasis II
Horemheb
Nefertiti
Pope Nicholas III


In [145]:
for node in ranking[-20:]:
    print(articles[node])

Viktor Dotsenko
Ste Curran
Wayne Smith (Chief Statistician of Canada)
Natasha Alexandra
Inayatullah (Guantanamo detainee 10029)
Martin Hanlin
Troy Dunn
Margaret O'Leary
David Getches
Richard Orr
Tony Medina
Donald N. Sills
Dave Glinka
Bb Manga
Roxane Berard
Josateki Nawalowalo
Charles W. Curtis
Arnold J. Levine
Alan Klingenstein
Don Rondo


![wiki](https://static1.squarespace.com/static/5a4fb547b0786969790f230b/t/5a626c17ec212d0fa48d9fe5/1516399656615/Wikipedia+Banner)